<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#6.6-Aprendizaje-por-diferencia-temporal:-SARSA-y-Q-Learning" data-toc-modified-id="6.6-Aprendizaje-por-diferencia-temporal:-SARSA-y-Q-Learning-1">6.6 Aprendizaje por diferencia temporal: SARSA y Q-Learning</a></span><ul class="toc-item"><li><span><a href="#Diferencia-temporal-como-suma-de-Monte-Carlo-y-programación-dinámica" data-toc-modified-id="Diferencia-temporal-como-suma-de-Monte-Carlo-y-programación-dinámica-1.1">Diferencia temporal como suma de Monte Carlo y programación dinámica</a></span></li><li><span><a href="#SARSA" data-toc-modified-id="SARSA-1.2">SARSA</a></span><ul class="toc-item"><li><span><a href="#Algoritmo" data-toc-modified-id="Algoritmo-1.2.1">Algoritmo</a></span></li></ul></li><li><span><a href="#Q-Learning" data-toc-modified-id="Q-Learning-1.3">Q-Learning</a></span><ul class="toc-item"><li><span><a href="#Algoritmo" data-toc-modified-id="Algoritmo-1.3.1">Algoritmo</a></span></li></ul></li><li><span><a href="#On-policy-versus-Off-policy" data-toc-modified-id="On-policy-versus-Off-policy-1.4"><em>On-policy versus Off-policy</em></a></span></li></ul></li></ul></div>

## 6.6 Aprendizaje por diferencia temporal: SARSA y Q-Learning
---

Anteriormente, hemos visto cómo métodos como MC permiten aprender la política óptima sin necesidad de conocer la dinámica del entorno (a diferencia de métodos como _Value Iteration_).

<br>Ahora bien, estos métodos requieren realizar episodios completos para obtener el **retorno real** antes de poder actualizar y realizar mejoras en la estimación de la función de valor.

<br>En 1988, Richard S. Sutton mostró como el método MC puede usar la diferencia entre predicciones temporalmente sucesivas combinando las ventajas de la programación dinámica y el método MC.

### Diferencia temporal como suma de Monte Carlo y programación dinámica
---

Recordemos que la programación dinámica usa la ecuación de Bellman para calcular el valor de un estado (suma de la recompensa inmediata y el valor descontado del siguiente estado). Es decir, para calcular el valor de un estado, no tenemos que esperar al final del episodio (como en MC), sino que podemos estimar el valor de un estado basándonos el valor estimado del siguiente estado:

<br>

<center>$V_\pi(s)=\sum_{s'}P(s'|s,a)\cdot[R(s,a,s') + \gamma \cdot V_\pi(s')]$</center>

<br>Esto es lo que se denomina **bootstrapping**, es decir, estimar en base a otra estimación previa.

<br>El problema de la programación dinámica es que, en entornos estocásticos, necesitamos conocer la dinámica del modelo del entorno, es decir, la probabilidad de transición **P(s'|s,a)**

<br>En cambio, el método MC es _model\_free_. Su desventaja es que, para obtener las estimaciones de los valores de los estados y las acciones, necesitamos esperar hasta el final del episodio y, si la tarea es continua (no episódica), no podremos aplicar dicho método.

<br>Una característica de MC es que tiene propiedades de convergencia bastante sólidas, porque actualiza la estimación de la función de valor con el entorno real, que es una **estimación no sesgada** de la función de valor real.

<br>Sin embargo, aunque los retornos reales obtenidos son estimaciones bastante precisas, también es una **estimación de alta varianza** de la función de valor real. Esto se debe a que los retornos reales acumulan muchos eventos aleatorios en la misma trayectoria. Asi, MC puede ser muy ineficiente en la obtención de las muestras de datos para calcular las funciones de valor. Toda esa aleatoriedad se convierte en ruido que sólo puede aliviarse con muchas trayectorias y muestras de retornos reales.

<br>Una forma de disminuir este problema consiste, en lugar de utilizar el entorno real, estimar un retorno y aprender de los episodios incompletos. Y este es precisamente el enfoque de diferencia temporal (TD). Entonces, al igual que en PD, se emplea _bootstrapping_ para no tener que esperar al final del episodio para calcular los valores de los estados.

<br>Y, de igual modo, para el valor de los pares estado-acción. Recordemos que en MC, para obtener una política óptima, la ecuación de actualización después de un **episodio completo** es:

<br>

<center>$Q(s_{t},A_{t})\leftarrow Q(s_{t},A_{t}) + \alpha(G_{t}-Q(s_{t},A_{t}))$</center>

<br>La única diferencia entre los métodos TD y MC es que **los métodos TD actualizan la tabla Q en cada _time step_ en lugar de esperar al final del episodio**

<br>Esta mejora tuvo un gran impacto en el avance de los métodos de RL y, el aprendizaje TD es el precursor de técnicas como SARSA o Q-Learning, que son la base de muchos métodos actuales.


### SARSA
---

Cuando un agente comienza a interactuar con un entorno, recibe el primer estado **S<sub>0</sub>**. A continuación utiliza la política para elegir la acción **A<sub>0</sub>**. Inmediatemente después recibe una recompensa **R<sub>1</sub>** y el siguiente estado **S<sub>1</sub>**. Luego, usando **la misma política** elige la acción **A<sub>1</sub>**, resultando en la secuencia siguiente que da nombre al algoritmo:

<br>

<center>$S_{0},A_{0},R_{1},S_{1},A_{1}$</center>

<br>Una vez obtenida esta secuencia del episodio, en lugar de esperar al final del mismo, actualiza la tabla Q para el par **Q(S<sub>0</sub>,A<sub>0</sub>)**. Sin embargo, en lugar de emplear el retorno (G) como una estimación para actualizar Q(S<sub>0</sub>,A<sub>0</sub>), se usa la suma de la recompensa inmediata **R<sub>1</sub>** y el valor descontado (x _gamma_) del siguiente par estado-acción Q(S<sub>1</sub>,A<sub>1</sub>):

<br>
<center>$Q(S_{0}.A_{0}) \leftarrow Q(S_{0}.A_{0}) + \alpha [R_{1} + \gamma \cdot Q(S_{1},A_{1}) - Q(S_{0}.A_{0})]$</center>

<br>que, de forma general, sería:

<br>
<center>$Q(S_{t}.A_{t}) \leftarrow Q(S_{t}.A_{t}) + \alpha [R_{t+1} + \gamma \cdot Q(S_{t+1},A_{t+1}) - Q(S_{t}.A_{t})]$</center>

<br>
<center><img src="trayectoria-SARSA.png"/></center>

#### Algoritmo
---

<center><img src="algo-SARSA.png" width="600"/></center>

### Q-Learning
---

También conocido como SARSAMAX, es exactamente como SARSA con la diferencia de que no sigue un política para determinar la siguiente acción, sino que elige la acción que tiene el valor de Q máximo.

<br>Este método comienza igual que SARSA, en el que el agente recibe el estado inicial S<sub>0</sub> y utiliza la política para elegir la acción A<sub>0</sub>. Inmediatamente después recibe un recompensa R<sub>1</sub> y el siguiente estado S<sub>1</sub>. Pero ahora, en lugar de usar la misma política para elegir la siguiente acción A<sub>1</sub>, se usa una **política _greedy_**, es decir, se seleccionará la acción que maximiza el valor de Q(s,a) para una acción determinada.

<br>Entonces, la ecuación de actualización para Q-Learning será:

<br>
<center>$Q(S_{0}.A_{0}) \leftarrow Q(S_{0}.A_{0}) + \alpha [R_{1} + \gamma \cdot max_{a} Q(S_{1},a) - Q(S_{0}.A_{0})]$</center>

<br>que, de forma general, sería:

<br>
<center>$Q(S_{t}.A_{t}) \leftarrow Q(S_{t}.A_{t}) + \alpha [R_{t+1} + \gamma \cdot Q(S_{t+1},a) - Q(S_{t}.A_{t})]$</center>

<br>Esta ecuación se basa en el hecho de que la acción _greedy_ correspondiente a un estado es justo la acción que maximiza el valor de ese estado.

<br>En este método, después de actualizar el valor del par estado-acción (S<sub>0</sub>,A<sub>0</sub>), seleccionamos A<sub>1</sub> usando la política **_épsilon-greedy_** correspondiente a los valores de Q que acabamos de actualizar.

<br>
<center><img src="trayectoria-QL.png"/></center>

#### Algoritmo
---

<center><img src="algo-QL.png" width="600"/></center>

### _On-policy versus Off-policy_
---

En el aprendizaje **_on-policy_**, la tabla Q se aprende a partir de acciones que se realizan siguiendo la política **π** que guía al agente y que es la que queremos optimizar.

<br>En el aprendizaje **_off-policy_**, la tabla Q se aprende seleccionando acciones con estrategias diferentes a las marcadas por la política del agente.

<br>Decimos que SARSA es _on-policy_ porque la política _epsilon-greedy_ que se evalúa y se va mejorando para obtener la política óptima es la misma que se utiliza para aprender la tabla Q.

<br>Decimos que QL es _off-policy_ porque la política _epsilon-greedy_ que se evalúa y guía al agente es diferente de la política _greedy_ que se utiliza para aprender la tabla Q.